In [17]:
import logging
import random
import heapq
import numpy as np
from typing import Callable

logging.basicConfig(format="%(message)s", level=logging.INFO)

In [18]:
class State:
    def __init__(self, data: np.ndarray):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data.tolist())

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

class PriorityQueue:
    """A basic Priority Queue with simple performance optimizations"""

    def __init__(self):
        self._data_heap = list()
        self._data_set = set()

    def __bool__(self):
        return bool(self._data_set)

    def __contains__(self, item):
        return item in self._data_set

    def push(self, item, p=None):
        assert item not in self, f"Duplicated element"
        if p is None:
            p = len(self._data_set)
        self._data_set.add(item)
        heapq.heappush(self._data_heap, (p, item))

    def pop(self):
        p, item = heapq.heappop(self._data_heap)
        self._data_set.remove(item)
        return item

In [24]:
def unit_cost(a):
    return len(a)

def priority_function(state):
    state_list = state.data.tolist()
    return sum(len(s) for s in state_list)

def possible_blocks(block, state):
    pb = block.copy()
    for s in state:
        pb.remove(s)
    return pb

def goal_test(state, N):
    state_list = state.data.tolist()
    new_list = set()
    for lst in state_list:
        for s in lst:
            new_list.add(s)
    return new_list == set(range(N))

def h(state, N):
    state_list = state.data.tolist()
    new_list = set()
    for lst in state_list:
            new_list.add(s for s in lst) 
    return len(range(N)) - len(new_list)


In [25]:
def problem(N, seed=42):
    random.seed(seed)
    blocksNotUnique = [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]
    retval = np.unique(np.array(blocksNotUnique, dtype=object))
    return retval.tolist()


def define_new_state(state, a):
    state.append(a)
    return State(np.array(state, dtype=object))

In [26]:
def search(
    blocks,
    initial_state: State,
    goal_test: Callable,
    parent_state: dict,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
):
    frontier = PriorityQueue()
    parent_state.clear()
    state_cost.clear()
    state = initial_state
    parent_state[state] = None
    state_cost[state] = 0
    while state is not None and not goal_test(state, N):
        for a in possible_blocks(blocks, state.data.tolist()):
            # print("state:", state)
            # print("possible blocks:", possible_blocks(blocks, state.data.tolist()))
            new_state = define_new_state(state.data.tolist(), a)
            # print("new state", new_state)
            cost = unit_cost(a)
            if new_state not in state_cost and new_state not in frontier:
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                # logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
                # print(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                parent_state[new_state] = state
                state_cost[new_state] = state_cost[state] + cost
                # logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
                # print(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    path = list()
    s = state
    while s:
        path.append(s.copy_data().tolist())
        s = parent_state[s]
    # logging.info(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    print(f"Initial blocks : {blocks}")
    print(f"Found a solution in {len(path):,} steps; visited {len(state_cost):,} states")
    for k,v in enumerate(reversed(path)):
        print(f"Step {k} : {v}")
    print(f"Step {len(path) - 1} is Solution")

    return list(reversed(path))

In [27]:
parent_state = dict()
state_cost = dict()
for N in [20]:
    blocks = problem(N, seed = 42)
    print(f"For N: {N}")
    print(blocks)
    final = search(
        sorted(blocks, key=lambda l: len(l)),
        State(np.array([])),
        goal_test=goal_test,
        parent_state=parent_state,
        state_cost=state_cost,
        priority_function=lambda s: state_cost[s] + h(s, N),
        unit_cost=unit_cost
    )

For N: 20
[[0, 1, 2, 3, 5, 7, 14, 17], [0, 1, 2, 3, 6, 13, 17, 18], [0, 1, 2, 7], [0, 1, 3, 7, 9, 10, 11, 15], [0, 3, 5, 8, 9, 10, 13, 14, 17], [0, 5, 7, 8, 13, 14, 17, 18], [0, 5, 11, 16, 17], [0, 6, 16, 17, 19], [1, 2, 9, 7], [1, 3, 4, 5], [1, 3, 8, 11, 14, 19], [1, 3, 13, 14], [2, 3, 4, 6, 8, 10], [2, 3, 7, 10, 14, 16], [2, 3, 9, 11, 12, 17, 18, 19], [2, 5, 7, 8, 12, 14, 17, 19], [2, 6, 8, 10, 12, 15, 18], [2, 8, 12, 13, 14, 16, 17, 19], [3, 5, 6, 7, 8, 11, 12, 14], [3, 6, 7, 10, 14, 17], [3, 6, 7, 13, 15], [4, 5, 8, 13, 15, 16, 17, 19], [4, 6, 15, 17, 18], [4, 7, 8, 14, 17, 18], [4, 7, 11, 12, 15, 16, 18], [5, 7, 8, 13, 14], [6, 9, 11, 12, 17], [8, 4, 7], [8, 16, 5], [16, 9, 19, 6], [16, 10, 2, 7], [17, 10, 1, 7], [17, 18, 7], [18, 2, 15]]


KeyboardInterrupt: 